# EXAMPLE 1: 3 players, 2 actions

In [1]:
import numpy as np
from sage.all import *
from gtnash.game.hypergraphicalgame  import *
from gtnash.util.irda import *
from gtnash.util.polynomial_complementary_problem import *
from gtnash.solver.wilsonalgorithm import *

def present_node(current_node,y_values):
    print(f"Level: {current_node.level}")
    var_pl=[]
    var_notpl=[]
    for (n,i) in current_node.pcp.couple_to_x.keys():
        if n>current_node.level:
            var_notpl+=[current_node.pcp.couple_to_x[(n,i)]]
        else:
            var_pl+=[current_node.pcp.couple_to_x[(n,i)]]
    print(f"Variables of players at the current level: {var_pl}")
    print(f"Variables of players above the current level:{var_notpl}")
    print("Polynomials:")
    for pol in current_node.pcp.couple_to_poly.keys(): 
        print(" "+str(pol)+": "+str(current_node.pcp.couple_to_poly[pol].subs(y_values)))

    print(f"Set Z: {current_node.set_z}")
    print(f"Set W: {current_node.set_w}")
    print(f"Coordinate of the node: {current_node.coordinates}")

    print("Equations:")
    for e in range(len(current_node.sys_S.equations)):
        if not current_node.sys_S.equations[e].subs(y_values)==0:
            print(f" {current_node.sys_S.equations[e].subs(y_values)} = 0")
    print("Inequations:")
    for (n,i) in current_node.pcp.couple_to_x.keys():
        if not (n,i) in current_node.set_z:
            print(f" {current_node.pcp.couple_to_x[n,i]} >= 0")
    for e in range(len(current_node.sys_S.inequations)):
        print(f" {current_node.sys_S.inequations[e].subs(y_values)} >=0 ")


## Definition of the game:

Before creating the PCP of a game, we first compute its disutility:

In [ ]:
players_actions = [[0, 1], [0, 1], [0, 1]]
utilities=[[6,7,0,5,1,4,2,3],[0,3,7,4,6,5,1,2],[4,7,4,0,3,2,6,1]]
current_nfg=NFG(players_actions,utilities)
print("Utilities of the game")
print(current_nfg)

label = []
for i in range (len(current_nfg.players_actions)):
    label.append('Ax'.replace("x", str(i))) #'A _i'
for i in range (len(current_nfg.players_actions),2*(len(current_nfg.players_actions))):
    label.append('ax'.replace("x", str(i-len(current_nfg.players_actions)))) #'U _i'

tableau_final = np.concatenate((np.mat(label), np.concatenate( (current_nfg.joint_actions,current_nfg.disutilities.astype(int).astype(str)),axis=1)),axis=0)
print("Disutilities of the game:")
print(tableau_final)

## PCP: Polynomials creation

Using the disutilities, we compute the polynomials associated to each couple (player, action) to create the PCP associated to this game.

Here the variables yn_i will all be equals to 1.

In [ ]:
pcpglobal = PolynomialComplementaryProblem(current_nfg)
print(pcpglobal)
#MAnual print after removing the yn_g variables?

## Choosing an arbitrary joint strategy
Before using the path following algorthim an joint strategy must be chosen between among all the possible joint strategy. Meaning that for this example we have the choice between:
* (0,0,0)
* (0,0,1)
* (0,1,0)
* (0,1,1)
* (1,0,0)
* (1,0,1)
* (1,1,0)
* (1,1,1)

Here we will choose omega0=(0,0,0)

## Computing the first node:


In [ ]:
omega0 = pcpglobal.omega0
xpairs_val = {}
for (n,i) in pcpglobal.couple_to_x.keys():
    if omega0[n]==i:
        xpairs_val[(n,i)] = 1
    else:
        xpairs_val[(n,i)] = 0

y_values={pcpglobal.ring("y0_0"):1, pcpglobal.ring("y1_0"):1, pcpglobal.ring("y2_0"):1}
print("The static value given to each variable xn_i for a couple (n,i) when current level< n are the following:")
print("xpairs_val:\n",xpairs_val)


# check first node computation:
node0 = first_node(pcpglobal,xpairs_val)[0]
#print("The corresponding first node is:\n",node0)
print("\nThe corresponding first node is:\n")
print(f"Level: {node0.level}")

var_pl=[]
var_notpl=[]
for (n,i) in node0.pcp.couple_to_x.keys():
    if n>node0.level:
        var_notpl+=[node0.pcp.couple_to_x[(n,i)]]
    else:
        var_pl+=[node0.pcp.couple_to_x[(n,i)]]
print(f"Variables of players at the current level: {var_pl}")
print(f"Variables of players above the current level:{var_notpl}")
print("Polynomials:")
for pol in node0.pcp.couple_to_poly.keys(): 
    print(" "+str(pol)+": "+str(node0.pcp.couple_to_poly[pol].subs(y_values)))
    
print(f"Set Z: {node0.set_z}")
print(f"Set W: {node0.set_w}")
print(f"Coordinate of the node: {node0.coordinates}")


## Complementary node at the level 0

For this node we have:
* Z=[(0,1)]
* W=[(0,0)]


Meaning that the node is compelmentary.

Given that the node is complementary at the level 0, we ascend the to level 1.


To reach the initial node at the level 1 for the node we found, we follow the arc (1,0) almost complementary where 
* Z=[(0,1),(1,1)]
* W=[(0,0)]

Because the initial joint strategy was (0,0,0) we initial must have the couple (1,1) in Z.

Considering that we are doing a lifting procedure we have two possible node we can reach, one for each possible couple (1,i) we can add to the set W:
* Z=[(0,1),(1,1)]
* W=[(0,0),(1,0)]

Or

* Z=[(0,1),(1,1)]
* W=[(0,0),(1,1)]

The node must be feasible, meaning that every value of the coordinate (variables xn_i) as well as the value of the polynomials An_i are >=0  


In [ ]:
node1=node0.lift(pcpglobal,xpairs_val)[0]

print("\nThe feasible initial node is:\n")
present_node(node1,y_values)

#print(node1)

## Arc traversal at the level 1

The initial node is:
* Z=[(0,1),(1,1)]
* W=[(0,0),(1,1)]

The node found is almost complementary meaning that we must follow the arc that "leaves" from the node.

Given that the couple that "entered" a set was (1,1) in W, to try an reach a complementary node throught the (1,0) almost complementary path we remove (1,1) from the set Z meaning that we will follow the arc:

* Z=[(0,1)]
* W=[(0,0),(1,1)]


Which in turn means that the couple that can enter Z are :
* (1,0)

The couple (0,0) can't enter Z because it would correspond to a strategy where the player associated to 0 play no strategy.

And enter W are:
* (0,1)
* (1,0)

Which means there is 3 possible node we can reach:


* Z=[(0,1),(1,0)]
* W=[(0,0),(1,1)]

,

* Z=[(0,1)]
* W=[(0,0),(1,1),(0,1)]

And

* Z=[(0,1)]
* W=[(0,0),(1,1),(1,0)]


In [ ]:
# Compute the next arc:
nodepcp = node1.pcp
level = node1.level
all_couples = set([(n,i) for (n,i) in nodepcp.couple_to_x.keys() if n<=level])
(arc_set_z,arc_set_w) = node1.compute_arc(node0)    
poss_new_z = [(m,j) for (m,j) in all_couples.difference(set(node1.set_z))]
poss_new_w = [(m,j) for (m,j) in all_couples.difference(set(node1.set_w))]
print("Arc:")
print(f"Set Z: {arc_set_z}")
print(f"Set W: {arc_set_w}")
print("Possible (n,i)")
print(f"In Z: {poss_new_z}")
print(f"In W: {poss_new_w}")

print("---------------------------------------------------------------------------------------------")

z = poss_new_z.pop(0)
loc_set_z = arc_set_z+[z]
loc_set_w = arc_set_w
print(" ")
print("Node currently considered:" )
print(f" Z: {loc_set_z}")
print(f" W: {loc_set_w}")

equations = [nodepcp.ring(nodepcp.couple_to_x[(n,i)]) for (n,i) in loc_set_z] # was self.ring
dico={}
for e in range(len(equations)):
    dico[equations[e]]=0
subpcp = nodepcp.substitute(dico,level)
# Zero polynomials equation:
#print("SubPCP:",subpcp)
equations += [subpcp.couple_to_poly[(n,i)].subs(y_values) for (n,i) in loc_set_w if n<=level]
# Add equations defining the y
#equations += [nodepcp.couple_to_poly_y[(n,g)] for (n,g) in nodepcp.couple_to_poly_y.keys()]
# Add fixed variables values for n>level
equations += [nodepcp.ring(nodepcp.couple_to_x[(n,i)])-xpairs_val[(n,i)] for (n,i) in nodepcp.couple_to_x.keys() if n>level]

print("Equations associated to the node")
for eq in equations:
    print(f" {eq}=0")
current_id=ideal(equations+[nodepcp.couple_to_poly_y[(n,g)] for (n,g) in nodepcp.couple_to_poly_y.keys()])
print("Dimension of the ideal")
print(current_id.dimension())
print("Groebner basis")
grob_basis=current_id.groebner_basis()
for eq in grob_basis:
    if not str(eq)[0]=='y':
        print(f" {eq} = 0")
print("Computed variety(ies) and value of the polynomials")
varieties=current_id.variety()
for i_v,v in enumerate(varieties):
    print(f" Variety {i_v}:")
    for x in nodepcp.set_x:
        print(f" {x} = {v[nodepcp.ring(x)]}")
    for (n,i) in nodepcp.couple_to_poly.keys():
        print(f" A{n}_{i}-1 = {nodepcp.couple_to_poly[(n,i)].subs(v)}")
print("At least one negative value: Not feasible")

print("---------------------------------------------------------------------------------------------")
        
z = poss_new_z.pop(0)
loc_set_z = arc_set_z+[z]
loc_set_w = arc_set_w
print(" ")
print("Node currently considered:" )
print(f" Z: {loc_set_z}")
print(f" W: {loc_set_w}")
print("Skip: Unfeasible node")

print("---------------------------------------------------------------------------------------------")

w = poss_new_w.pop(0)
loc_set_z = arc_set_z
loc_set_w = arc_set_w+[w]
print(" ")
print("Node currently considered:" )
print(f" Z: {loc_set_z}")
print(f" W: {loc_set_w}")


equations = [nodepcp.ring(nodepcp.couple_to_x[(n,i)]) for (n,i) in loc_set_z] # was self.ring
dico={}
for e in range(len(equations)):
    dico[equations[e]]=0
subpcp = nodepcp.substitute(dico,level)
# Zero polynomials equation:
#print("SubPCP:",subpcp)
equations += [subpcp.couple_to_poly[(n,i)].subs(y_values) for (n,i) in loc_set_w if n<=level]
# Add equations defining the y
#equations += [nodepcp.couple_to_poly_y[(n,g)] for (n,g) in nodepcp.couple_to_poly_y.keys()]
# Add fixed variables values for n>level
equations += [nodepcp.ring(nodepcp.couple_to_x[(n,i)])-xpairs_val[(n,i)] for (n,i) in nodepcp.couple_to_x.keys() if n>level]

print("Equations associated to the node")
for eq in equations:
    print(f" {eq}=0")
current_id=ideal(equations+[nodepcp.couple_to_poly_y[(n,g)] for (n,g) in nodepcp.couple_to_poly_y.keys()])
print("Dimension of the ideal")
print(current_id.dimension())
print("Groebner basis")
grob_basis=current_id.groebner_basis()
for eq in grob_basis:
    if not str(eq)[0]=='y':
        print(f" {eq} = 0")
print("Computed variety(ies) and value of the polynomials")
varieties=current_id.variety()
for i_v,v in enumerate(varieties):
    print(f" Variety {i_v}:")
    for x in nodepcp.set_x:
        print(f" {x} = {v[nodepcp.ring(x)]}")
    for (n,i) in nodepcp.couple_to_poly.keys():
        print(f" A{n}_{i}-1 = {nodepcp.couple_to_poly[(n,i)].subs(v)}")
        
print("Feasible node")
        
print("---------------------------------------------------------------------------------------------")
w = poss_new_w.pop(0)
loc_set_z = arc_set_z
loc_set_w = arc_set_w+[w]
print(" ")
print("Node currently considered:" )
print(f" Z: {loc_set_z}")
print(f" W: {loc_set_w}")


equations = [nodepcp.ring(nodepcp.couple_to_x[(n,i)]) for (n,i) in loc_set_z] # was self.ring
dico={}
for e in range(len(equations)):
    dico[equations[e]]=0
subpcp = nodepcp.substitute(dico,level)
# Zero polynomials equation:
#print("SubPCP:",subpcp)
equations += [subpcp.couple_to_poly[(n,i)].subs(y_values) for (n,i) in loc_set_w if n<=level]
# Add equations defining the y
#equations += [nodepcp.couple_to_poly_y[(n,g)] for (n,g) in nodepcp.couple_to_poly_y.keys()]
# Add fixed variables values for n>level
equations += [nodepcp.ring(nodepcp.couple_to_x[(n,i)])-xpairs_val[(n,i)] for (n,i) in nodepcp.couple_to_x.keys() if n>level]

print("Equations associated to the node")
for eq in equations:
    print(f" {eq}=0")
current_id=ideal(equations+[nodepcp.couple_to_poly_y[(n,g)] for (n,g) in nodepcp.couple_to_poly_y.keys()])
print("Dimension of the ideal")
print(current_id.dimension())
print("No solution, unfeasible node")

In [ ]:
node2=node1.traverse(node0, xpairs_val)[0]


print("\nThe almost complementary node is:\n")
present_node(node2,y_values)

The node found is 
* Z=[(0,1)]
* W=[(0,0),(1,1),(0,1)]

This node is almost complementary meaning that we must continue the execution

## Second Arc traversal at the level 1

Given that the couple (0,1) has entered the set W, the arc we will follow is the one where we remove the same couple from the set Z:

* Z=[]
* W=[(0,0),(1,1),(0,1)]

The node we can reach are the following:

* Z=[(0,0)]
* W=[(0,0),(1,1),(0,1)]

,

* Z=[(1,0)]
* W=[(0,0),(1,1),(0,1)]

,

* Z=[(1,1)]
* W=[(0,0),(1,1),(0,1)]

and

* Z=[]
* W=[(0,0),(1,1),(0,1),(1,0)]



In [ ]:
node3=node2.traverse(node1, xpairs_val)[0]


print("\nThe almost complementary node is:\n")
present_node(node3,y_values)

The node found is 
* Z=[]
* W=[(0,0),(1,1),(0,1),(1,0)]

This node is complementary meaning that we can go to the next level

## Lifing procedure from the level 1 to 2

Considering the joint strategy chosen initialy we will follow the arc

* Z=[(2,1)]
* W=[(0,0),(1,1),(0,1),(1,0)]

Meaning that we can reach 2 initial nodes:

* Z=[(2,1)]
* W=[(0,0),(1,1),(0,1),(1,0),(2,0)]

or 

* Z=[(2,1)]
* W=[(0,0),(1,1),(0,1),(1,0),(2,1)]


In [ ]:
node4=node3.lift(pcpglobal, xpairs_val)[0]
    
print("\nThe feasible initial node is:\n")
present_node(node4,y_values)

## Complementary node found

* Z=[(2,1)]
* W=[(0,0),(1,1),(0,1),(1,0),(2,0)]

The node found is complementary and we are at the last level, meaning that this node is a solution to the PCP which correspond to mixed nash equilibrium of the corresponding game.

The coordinate of the nodes are:

In [ ]:
for (n,i) in pcpglobal.couple_to_x.keys():
    print(f"{pcpglobal.couple_to_x[(n,i)]}={node4.coordinates[pcpglobal.couple_to_x[(n,i)]]}={node4.coordinates[pcpglobal.couple_to_x[(n,i)]].radical_expression()}")

Which when normalized give us the following mixed strategy:

In [ ]:
mixed_strat=node4.normalized_strategy()
for n in mixed_strat.keys():
    print(f"Player {n}: {mixed_strat[n]}")